In [2]:
import pandas as pd
from itertools import combinations

# Charger les données
credits_data = pd.read_csv("../data/credits.csv")
titles_data = pd.read_csv("../data/titles.csv")

titles_data["title"].fillna("Unknown", inplace=True)
# titles_data.dropna(subset=["imdb_score", "tmdb_score"], inplace=True)
# titles_data["tmdb_score"].dropna(inplace=True)


credits_data.head()
titles_data.head()
titles_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5850 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

In [3]:
id_to_title_dict = titles_data.dropna(subset=['title']).set_index('id')['title'].to_dict()

list(id_to_title_dict.items())[:5]


[('ts300399', 'Five Came Back: The Reference Films'),
 ('tm84618', 'Taxi Driver'),
 ('tm154986', 'Deliverance'),
 ('tm127384', 'Monty Python and the Holy Grail'),
 ('tm120801', 'The Dirty Dozen')]

In [4]:
# Regrouper les acteurs par film
grouped = credits_data.groupby('id')['name'].apply(list)

# Créer des paires d'acteurs pour chaque film
actor_pairs = []
for actors in grouped:
    for pair in combinations(actors, 2):
        actor_pairs.append(pair)

# Afficher les premières paires pour vérifier
actor_pairs[:10]


[('Luna Wedler', 'Jannis Niewöhner'),
 ('Luna Wedler', 'Milan Peschel'),
 ('Luna Wedler', 'Edin Hasanović'),
 ('Luna Wedler', 'Anna Fialová'),
 ('Luna Wedler', 'Marlon Boess'),
 ('Luna Wedler', 'Victor Boccard'),
 ('Luna Wedler', 'Fleur Geffrier'),
 ('Luna Wedler', 'Aziz Dyab'),
 ('Luna Wedler', 'Mélanie Fouché'),
 ('Luna Wedler', 'Elizaveta Maximová')]

In [7]:
import plotly.graph_objects as go
import networkx as nx


def create_actor_network(df, movie_ids, scale_factor=0.5):
    """
    Crée un graphe de réseau pour les acteurs qui ont joué dans les films spécifiés.

    Args:
    df (pd.DataFrame): DataFrame contenant les données des acteurs et des films.
    movie_ids (list): Liste des identifiants des films à inclure dans le graphe.
    scale_factor (float): Dans ce code, j'ai ajouté un facteur de mise à l'échelle de 0.5 pour la taille des nœuds, avec une taille de base de 5.
    base_node_size (int): Vous pouvez ajuster ce facteur de mise à l'échelle pour obtenir la représentation visuelle souhaitée. Plus la valeur est élevée, plus la différence de taille entre les nœuds sera perceptible.

    Returns:
    plotly.graph_objs._figure.Figure: Un objet Figure Plotly représentant le graphe de réseau.
    """

    # Filtrage des données pour les films spécifiés
    filtered_df = df[df["id"].isin(movie_ids)]

    # Création d'un graphe vide
    G = nx.Graph()

    # Ajout des nœuds (acteurs) et des arêtes (relations entre acteurs) dans le graphe
    for movie_id in movie_ids:
        actors = filtered_df[filtered_df["id"] == movie_id]["name"].unique()
        for actor in actors:
            G.add_node(actor)
        for i in range(len(actors)):
            for j in range(i + 1, len(actors)):
                G.add_edge(actors[i], actors[j])

    # Positionnement des nœuds
    pos = nx.spring_layout(G)

    # Création des arêtes
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Création des nœuds
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    # Création de la figure Plotly
    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color="#888"),
        hoverinfo="none",
        mode="lines",
    )

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers",
        hoverinfo="text",
        text=list(G.nodes()),
        marker=dict(
            showscale=True,
            colorscale="YlGnBu",
            size=10,
            color=[],
            colorbar=dict(
                thickness=15,
                title="Nombre de Connexions",
                xanchor="left",
                titleside="right",
            ),
            line_width=2,
        ),
    )

    # Ajout du nombre de connexions comme attribut de couleur pour les nœuds
    node_adjacencies = []

    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))

    node_trace.marker.color = node_adjacencies
    node_trace.marker.size = [adj * scale_factor for adj in node_adjacencies]

    # TODO: Extraire cette logique dans une fonction séparée pour plus de clarté.
    # Créer un dictionnaire pour compter les connexions pour chaque acteur
    actor_connections = {actor: len(list(G.neighbors(actor))) for actor in G.nodes()}
    # Trier les acteurs par nombre de connexions
    sorted_actors = sorted(actor_connections.items(), key=lambda x: x[1], reverse=True)

    # Création de la figure finale
    fig = go.Figure(
        data=[edge_trace, node_trace],
        layout=go.Layout(
            title="<br>Réseau des acteurs dans les films/séries sélectionnés",
            titlefont_size=16,
            showlegend=False,
            hovermode="closest",
            margin=dict(b=20, l=5, r=5, t=40),
            annotations=[
                dict(
                    text="A chaque sélection de films/séries, la liste est re-calculé avec seulement les acteurs en commun.",
                    showarrow=False,
                    xref="paper",
                    yref="paper",
                    x=0.005,
                    y=-0.002,
                )
            ],
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        ),
    )

    return fig, sorted_actors


# Exemple d'utilisation de la fonction avec quelques identifiants de films
# example_movie_ids = ["tm84618", "tm155787"]  # Id du film "Taxi Driver" pour tester
# fig, sorted_actors = create_actor_network(credits_data, example_movie_ids, 0.09)
# fig.show()

In [5]:
dropdown_options = [
    {'label': title, 'value': str(movie_id)}
    for movie_id, title in sorted(id_to_title_dict.items(), key=lambda item: item[1])
]

i = 0
for options in dropdown_options:
  i += 1
  if(options['value'] == "tm155787"):
    print(options, i)


{'label': 'GoodFellas', 'value': 'tm155787'} 1894


In [6]:
def find_common_actors_movies(selected_movie_ids, df):
    if not selected_movie_ids:
        return df["id"].unique().tolist()

    selected_actors = df[df["id"].isin(selected_movie_ids)]["name"].unique()
    common_movies = df[df["name"].isin(selected_actors)]["id"].unique()
    return common_movies.tolist()

In [9]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
from dash import callback_context


app = dash.Dash(__name__)

# Assurez-vous que les autres imports nécessaires sont toujours là (pandas, networkx, etc.)

# Vous pouvez reprendre la fonction create_actor_network ici (assurez-vous qu'elle est définie)

# Utilisez votre DataFrame ici
# Si vous avez déjà le DataFrame 'data' chargé, vous pouvez le réutiliser

# Obtenez une liste unique des identifiants de films
unique_movie_ids = credits_data[credits_data["id"].isin(titles_data["id"])][
    "id"
].unique()
dropdown_options = [
    {"label": title, "value": str(movie_id)}
    for movie_id, title in sorted(id_to_title_dict.items(), key=lambda item: item[1])
]


app.layout = html.Div(
    [
        dcc.Store(id="sorted-actors-store"),  # Stocke les données des acteurs triés
        html.H3("Top 10 acteurs les plus connectés"),
        dcc.Dropdown(id="top-actors-dropdown"),
        html.Div(
            id="film-selector-container",
            style={"display": "none"},
            children=[
                dcc.Dropdown(
                    id="movie-id-dropdown",
                    options=dropdown_options,
                    # Par défaut, on prend des films où on voit beaucoup Scorcese et De Niro.
                    value=[
                        dropdown_options[4433]["value"],
                        dropdown_options[1893]["value"],
                        dropdown_options[3429]["value"],
                    ],
                    multi=True,
                ),
            ],
        ),
        dcc.Slider(
            id="scale-factor-slider",
            min=0.1,
            max=0.25,
            step=0.01,
            value=0.1,
            marks={i / 10: str(i / 10) for i in range(1, 26, 1)},
            tooltip={"placement": "bottom", "always_visible": True},
        ),
        dcc.Graph(id="actor-network"),
    ]
)


@app.callback(
    Output("film-selector-container", "style"),
    [Input("top-actors-dropdown", "value")],
)
def toggle_film_selector_visibility(selected_actor):
    if selected_actor:
        # Si un acteur est sélectionné, masquer le bouton et la liste
        return {"display": "none"}
    else:
        # Sinon, les afficher
        return {"display": "block"}


@app.callback(
    [
        Output("actor-network", "figure"),
        Output("sorted-actors-store", "data"),  # Stocker les données dans dcc.Store
    ],
    [
        Input("movie-id-dropdown", "value"),
        Input("top-actors-dropdown", "value"),  # Sélection de l'acteur
        Input("scale-factor-slider", "value"),
    ],
)
def update_graph(selected_movie_ids, selected_actor, scale_factor):
    if selected_actor:
        actor_movies = credits_data[credits_data["name"] == selected_actor][
            "id"
        ].unique()
        fig, sorted_actors = create_actor_network(
            credits_data, actor_movies, scale_factor
        )
    elif selected_movie_ids:
        fig, sorted_actors = create_actor_network(
            credits_data, selected_movie_ids, scale_factor
        )
    else:
        fig = go.Figure()
        sorted_actors = []

    # fig, sorted_actors = create_actor_network(
    #     credits_data, selected_movie_ids, scale_factor
    # )
    # Stockez sorted_actors dans un format JSON compatible
    sorted_actors_json = [
        {"actor": actor, "connections": connections}
        for actor, connections in sorted_actors
    ]

    return fig, sorted_actors_json


@app.callback(
    Output("top-actors-dropdown", "options"), [Input("sorted-actors-store", "data")]
)
def update_actor_list(sorted_actors_data):
    if sorted_actors_data:
        sorted_actors = [
            (item["actor"], item["connections"]) for item in sorted_actors_data[:10]
        ]
        return [{"label": actor, "value": actor} for actor, _ in sorted_actors]
    return []


@app.callback(
    Output("movie-id-dropdown", "options"),
    [Input("movie-id-dropdown", "value")],  # Sélection des films
)
def update_movie_dropdown(selected_movie_ids):
    # Si des films sont sélectionnés, mettre à jour la liste en fonction des acteurs en commun
    if selected_movie_ids:
        common_movie_ids = find_common_actors_movies(selected_movie_ids, credits_data)
        new_options = [
            {"label": id_to_title_dict[movie_id], "value": movie_id}
            for movie_id in common_movie_ids
        ]
    # Si rien n'est sélectionné, afficher tous les films
    else:
        new_options = [
            {"label": id_to_title_dict[movie_id], "value": movie_id}
            for movie_id in id_to_title_dict
        ]

    return new_options


if __name__ == "__main__":
    app.run_server(debug=True)